<a href="https://colab.research.google.com/github/Maple127/public127.github.io/blob/main/chapter_linear-classification/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install d2l
import torch

# The Base Classification Model
:label:`sec_classification`

You may have noticed that the implementations from scratch and the concise implementation using framework functionality were quite similar in the case of regression. The same is true for classification. Since many models in this book deal with classification, it is worth adding functionalities to support this setting specifically. This section provides a base class for classification models to simplify future code.


In [2]:
import torch
from d2l import torch as d2l

## The `Classifier` Class


We define the `Classifier` class below. In the `validation_step` we report both the loss value and the classification accuracy on a validation batch. We draw an update for every `num_val_batches` batches. This has the benefit of generating the averaged loss and accuracy on the whole validation data. These average numbers are not exactly correct if the final batch contains fewer examples, but we ignore this minor difference to keep the code simple.


In [3]:
class Classifier(d2l.Module):  #@save
    """The base class of classification models."""
    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)
        self.plot('acc', self.accuracy(Y_hat, batch[-1]), train=False)

By default we use a stochastic gradient descent optimizer, operating on minibatches, just as we did in the context of linear regression.


In [4]:
@d2l.add_to_class(d2l.Module)  #@save
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)

## Accuracy

Given the predicted probability distribution `y_hat`,
we typically choose the class with the highest predicted probability
whenever we must output a hard prediction.
Indeed, many applications require that we make a choice.
For instance, Gmail must categorize an email into "Primary", "Social", "Updates", "Forums", or "Spam".
It might estimate probabilities internally,
but at the end of the day it has to choose one among the classes.

When predictions are consistent with the label class `y`, they are correct.
The classification accuracy is the fraction of all predictions that are correct.
Although it can be difficult to optimize accuracy directly (it is not differentiable),
it is often the performance measure that we care about the most. It is often *the*
relevant quantity in benchmarks. As such, we will nearly always report it when training classifiers.

Accuracy is computed as follows.
First, if `y_hat` is a matrix,
we assume that the second dimension stores prediction scores for each class.
We use `argmax` to obtain the predicted class by the index for the largest entry in each row.
Then we [**compare the predicted class with the ground truth `y` elementwise.**]
Since the equality operator `==` is sensitive to data types,
we convert `y_hat`'s data type to match that of `y`.
The result is a tensor containing entries of 0 (false) and 1 (true).
Taking the sum yields the number of correct predictions.


In [5]:
@d2l.add_to_class(Classifier)  #@save
def accuracy(self, Y_hat, Y, averaged=True):
    """Compute the number of correct predictions."""
    Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
    preds = Y_hat.argmax(axis=1).type(Y.dtype)
    compare = (preds == Y.reshape(-1)).type(torch.float32)
    return compare.mean() if averaged else compare

## Summary

Classification is a sufficiently common problem that it warrants its own convenience functions. Of central importance in classification is the *accuracy* of the classifier. Note that while we often care primarily about accuracy, we train classifiers to optimize a variety of other objectives for statistical and computational reasons. However, regardless of which loss function was minimized during training, it is useful to have a convenience method for assessing the accuracy of our classifier empirically.


## Exercises

1. Denote by $L_\textrm{v}$ the validation loss, and let $L_\textrm{v}^\textrm{q}$ be its quick and dirty estimate computed by the loss function averaging in this section. Lastly, denote by $l_\textrm{v}^\textrm{b}$ the loss on the last minibatch. Express $L_\textrm{v}$ in terms of $L_\textrm{v}^\textrm{q}$, $l_\textrm{v}^\textrm{b}$, and the sample and minibatch sizes.
1. Show that the quick and dirty estimate $L_\textrm{v}^\textrm{q}$ is unbiased. That is, show that $E[L_\textrm{v}] = E[L_\textrm{v}^\textrm{q}]$. Why would you still want to use $L_\textrm{v}$ instead?
1. Given a multiclass classification loss, denoting by $l(y,y')$ the penalty of estimating $y'$ when we see $y$ and given a probabilty $p(y \mid x)$, formulate the rule for an optimal selection of $y'$. Hint: express the expected loss, using $l$ and $p(y \mid x)$.


1

In [6]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

# 加载验证数据集
val_dataset = datasets.FashionMNIST(root='~/Datasets', train=False, transform=transforms.ToTensor(), download=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# 随机模型
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 10)
)
with torch.no_grad():
    for param in model.parameters():
        param.uniform_(-0.1, 0.1)

# 定义验证损失函数
def compute_losses(model, dataloader):
    total_loss = 0
    total_samples = 0
    quick_loss_sum = 0
    num_batches = len(dataloader)
    last_batch_loss = 0
    last_batch_size = 0

    for i, (X, y) in enumerate(dataloader):
        with torch.no_grad():
            y_hat = model(X)
            loss = F.cross_entropy(y_hat, y, reduction='none')
            batch_total = loss.sum().item()
            batch_mean = loss.mean().item()
            batch_size = y.shape[0]

            total_loss += batch_total
            total_samples += batch_size
            quick_loss_sum += batch_mean

            if i == num_batches - 1:
                last_batch_loss = batch_mean
                last_batch_size = batch_size

    Lv = total_loss / total_samples
    Lqv = quick_loss_sum / num_batches
    return Lv, Lqv, last_batch_loss, last_batch_size, total_samples

Lv, Lqv, lbv, B_last, N = compute_losses(model, val_loader)

# 近似表达式重建 Lv
Lv_estimate = ((N - B_last) / N) * Lqv + (B_last / N) * lbv

print(f"精确验证损失 Lv: {Lv:.4f}")
print(f"快速估计 Lqv:     {Lqv:.4f}")
print(f"最后一批损失 lbv: {lbv:.4f}")
print(f"表达式估计 Lv:    {Lv_estimate:.4f}")

100%|██████████| 26.4M/26.4M [00:01<00:00, 21.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 343kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 6.28MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 18.3MB/s]


精确验证损失 Lv: 2.3577
快速估计 Lqv:     2.3572
最后一批损失 lbv: 2.2662
表达式估计 Lv:    2.3571


2

In [7]:
import numpy as np

def run_multiple_trials(num_trials=50):
    Lv_list, Lqv_list = [], []
    for _ in range(num_trials):
        # 模型重新初始化
        for param in model.parameters():
            param.data.uniform_(-0.1, 0.1)
        Lv, Lqv, _, _, _ = compute_losses(model, val_loader)
        Lv_list.append(Lv)
        Lqv_list.append(Lqv)
    return np.mean(Lv_list), np.mean(Lqv_list)

mean_Lv, mean_Lqv = run_multiple_trials()
print(f"多次运行期望 E[Lv]:  {mean_Lv:.4f}")
print(f"多次运行期望 E[Lqv]: {mean_Lqv:.4f}")

多次运行期望 E[Lv]:  2.5504
多次运行期望 E[Lqv]: 2.5501


3

In [8]:
import torch

# 示例：类别数
num_classes = 5

# 定义损失矩阵 l(y, y')
# 示例：当 y ≠ y' 时惩罚为 1；对角线为 0
loss_matrix = torch.ones((num_classes, num_classes)) - torch.eye(num_classes)

# 模拟一组概率分布 p(y|x)，形状：[num_classes]
p_y_given_x = torch.tensor([0.1, 0.2, 0.3, 0.25, 0.15])

# 计算每个可能的 y' 的期望损失
expected_losses = torch.mv(loss_matrix.t(), p_y_given_x)

# 选择使期望损失最小的 y'
y_star = torch.argmin(expected_losses).item()

print("每个预测类别的期望损失：", expected_losses)
print("最优选择 y* =", y_star)

每个预测类别的期望损失： tensor([0.9000, 0.8000, 0.7000, 0.7500, 0.8500])
最优选择 y* = 2


[Discussions](https://discuss.d2l.ai/t/6809)
